### Preparation

In [1]:
#!pip install numpy
#!pip install pandas
#!pip install requests
#!pip install tqdm
#!pip install sqlalchemy

### Packages

In [2]:
import numpy as np
import pandas as pd
import requests
import os
from tqdm import tqdm
from sqlalchemy import create_engine

### Input

In [3]:
pairs    = ('EURUSD', 'XAUUSD', 'JPYUSD', 'CNYUSD', 'GBPUSD') #max 5 pairs for free API key

key       = '' #put your API key from https://polygon.io/dashboard/api-keys
capital   = 100 #$
UTC       = 7
SMA       = 28 #Simple Moving Average
timeframe = '1/minute'

#tradingfee = 0

### Download data

In [4]:
def download_data(symbol, timeframe, time):
    url = 'https://api.polygon.io/v2/aggs/ticker/C:' +pair+ '/range/' +timeframe+ '/' +time+ '?adjusted=true&sort=asc&limit=100000&apiKey=' +key
    resp = requests.get(url)
    frame = pd.DataFrame(resp.json()['results'])
    frame = frame.iloc[:, 2:7]
    frame.columns = ['Open','Close','High','Low','Time']
    frame.Time = pd.to_datetime(frame['Time'], unit = 'ms')
    frame[['Open', 'High', 'Low', 'Close']] = frame[['Open', 'High', 'Low', 'Close']].astype(float)
    
    #frame.index = pd.to_datetime(frame.index) + pd.Timedelta(hours=UTC)
    
    return frame

In [5]:
if os.path.exists("Pair.db"):
    os.remove("Pair.db")
    
engine = create_engine('sqlite:///Pair.db')

indTime  = timeframe.index('/')
dictTime = {'week':0, 'day':0 ,'hour':0, 'minute':0}
dictTime[timeframe[indTime+1:]] += int(timeframe[:indTime])*50000

end = pd.to_datetime('today').date()
start = end - pd.Timedelta(weeks=dictTime['week'], days=dictTime['day'], hours=dictTime['hour'], minutes=dictTime['minute'])
time = str(start)+'/'+str(end)

for pair in tqdm(pairs):
    download_data(pair, timeframe, time).to_sql(pair, engine, index=False)

100%|██████████| 5/5 [00:39<00:00,  7.87s/it]


### Main Program

In [6]:
class Backtest_SMA:
    
    def __init__(self, capital, SMA, pairs): 
        self.capital  = capital
        self.SMA      = SMA
        self.pairs   = pairs
            
    def method(self, pair):
        data = pd.read_sql(pair, engine).set_index('Time') 
        
        data['sma'] = data.Close.rolling(self.SMA).mean()
        data.dropna(inplace=True)
    
        data['Type'] = np.where(data['Close'].shift(1) > data['sma'].shift(1), 'long', 'short')
        
        data['change'] = data.Close.pct_change()
        
    
        data['long strategy']  = np.where(data['Close'] > data['sma'], 1, 0)
        data['Long positions'] = self.capital*(np.cumprod(data['long strategy'].shift(1)*data['change'] + 1))
        
        data['short strategy']  = np.where(data['Close'] > data['sma'], 0, -1)
        data['Short positions'] = self.capital*(np.cumprod(data['short strategy'].shift(1)*data['change'] + 1))
        
        data['Return SMA'] = data['Long positions']*data['Short positions']/self.capital
        data['Return hold']  = self.capital*np.cumprod(data['change']+1)
    
        data['win pos'] = np.where(data['Close']> data['sma'], 1, -1)
        data['win']     = np.cumsum(np.where(data['win pos'].shift(1)*data['change'] > 0, 1, 0))
        
        data.dropna(inplace=True)
        data['Winrate'] = data['win']/np.arange(1, len(data) + 1)
        
        return data.loc[:, ['Winrate', 'Type', 'Long positions', 'Short positions', 'Return SMA', 'Return hold']]
    
    
    def sorting(self):
        profits = []
        
        for pair in self.pairs:
            df = self.method(pair)
            
            ret = df[['Return SMA', 'Return hold']]
            ret.columns = ['Return SMA ($)', 'Return hold ($)']
            
            profits.append(ret.iloc[-1,:])

        frame = pd.DataFrame(profits, self.pairs)
        frame['edges ($)'] = frame['Return SMA ($)'] - frame['Return hold ($)']
        frame = frame.sort_values('edges ($)', ascending=False)
        return frame

    
    def output(self, pair):
        data = self.method(pair)
        
        print('Simulation of ' + '\033[1m' + str(pair) + '\033[0m' + ' runs from ' + '\033[1m' + str(data.index[0]) + '\033[0m' + ' to ' + '\033[1m' + str(data.index[-1]) + '\033[0m')
        print('Winrate Method: %5.2f' %(100*data['Winrate'][-1]) + '%\n')

        print('Return Hold                : $%5.2f' %(data['Return hold'][-1]))
        print('return from Long Positions : $%5.2f' %(data['Long positions'][-1]))
        print('return from Short Positions: $%5.2f' %(data['Short positions'][-1]))
        print('Return Method              : $%5.2f' %(data['Return SMA'][-1]))

        
    def journal(self, pair):
        if os.path.exists('Journal ' + pair + '.xlsx'):
            os.remove('Journal ' + pair + '.xlsx')
        
        self.method(pair).to_excel('Journal ' + pair + '.xlsx', engine='xlsxwriter') 

In [7]:
backtesting = Backtest_SMA(capital, SMA, pairs)

### Output

In [12]:
#backtesting.method('GBPUSD')

In [9]:
backtesting.sorting()

,Return SMA ($),Return hold ($),edges ($)
CNYUSD,86.726815,101.026570,-14.299754
EURUSD,76.144920,106.431364,-30.286444
GBPUSD,74.823411,105.828862,-31.005451
JPYUSD,40.594415,105.878017,-65.283603
XAUUSD,16.278608,106.454339,-90.175732


In [10]:
backtesting.output('GBPUSD')

Simulation of GBPUSD runs from 2022-10-16 22:05:00 to 2022-11-18 22:01:00
Winrate Method: 46.62%

Return Hold                : $105.83
return from Long Positions : $89.06
return from Short Positions: $84.02
Return Method              : $74.82


In [11]:
backtesting.journal('GBPUSD')